In [6]:
import pandas as pd
import bamboolib
import plotly
import warnings

from bunkatech.networks.networks import network_analysis

from bunkatech.semantics.extract_terms import extract_terms_df
from bunkatech.semantics.indexer import indexer
from bunkatech.semantics.get_embeddings import get_embeddings

from bunkatech.hierarchical_clusters import hierarchical_clusters
from bunkatech.hierarchical_clusters_label import hierarchical_clusters_label

from bunkatech.visualization.make_bubble import make_bubble
from bunkatech.visualization.sankey import make_sankey
from bunkatech.visualization.topics_treemap import topics_treemap

warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None

In [7]:
# Load Data
df = pd.read_csv('data/imdb_movies.csv', index_col = [0])

text_var = 'description'
index_var = "imdb"

#### Extract Terms from the a specific column

In [ ]:
terms = extract_terms_df(
        df,
        text_var=text_var,
        limit=2000,
        sample_size=3000,
        ngs=True, #ngrams
        ents=False, #entities
        ncs=True, #nouns
        drop_emoji=True,
        remove_punctuation=False,
        ngrams=(2, 2),
        include_pos=["NOUN", "PROPN", "ADJ"],
        include_types=["PERSON", "ORG"],
        language="en",
    )

#### Index terms extracted to the original corpus

In [9]:
# Index with the list of original words
df_terms = terms.copy()
df_terms['text'] = df_terms['text'].apply(lambda x: x.split(' | '))
df_terms = df_terms.explode('text').reset_index(drop=True)
list_terms = df_terms['text'].tolist()

df_indexed = indexer(df[text_var].tolist(), list_terms, db_path = '.')

# get the Main form and the lemma
df_indexed_full = pd.merge(df_indexed, df_terms, left_on = 'words', right_on = 'text')
df_indexed_full = df_indexed_full[['docs', 'lemma', 'main form', 'text']].copy()

df_indexed_full

,docs,lemma,main form,text
0,The Stoneman family finds its friendship with ...,civil war,Civil War,civil war
1,A manipulative woman and a roguish man conduct...,civil war,Civil War,civil war
2,"During the Spanish Civil War, an American alli...",civil war,Civil War,civil war
3,An American Civil War veteran embarks on a jou...,civil war,Civil War,civil war
4,A family saga covering several decades of West...,civil war,Civil War,civil war
...,...,...,...,...
9751,"Set in the small-town of Bareilly, Bitti is a ...",pritam vidrohi,Pritam Vidrohi,Pritam Vidrohi
9752,"Before NYC, college valedictorian Ben visits h...",loser dad,Loser dad,Loser dad
9753,"Before NYC, college valedictorian Ben visits h...",loser dad,Loser dad,loser dad
9754,"Before NYC, college valedictorian Ben visits h...",park loser,park loser,park loser


#### Cooccurrence Graph Categories

In [10]:
df_graph = df[['imdb', 'genre', 'country']].copy()

df_graph = df_graph.dropna()
df_graph['genre'] = df_graph['genre'].apply(lambda x: x.split(','))
df_graph = df_graph.explode('genre')

df_graph['country'] = df_graph['country'].apply(lambda x: x.split(','))
df_graph = df_graph.explode('country')


fig = network_analysis(
        df_graph,
        variables=["genre", "country"],
        key="imdb",
        top_nodes=500,
        global_filter=0.2,
        n_neighbours=5,
        method="force_directed",
        density=10,
        height_att=1500,
        width_att=1500,
        color="community",
        template="simple_white",
        edge_size=1.5,
        symbol="entity",
        save_path=".",
    )

plotly.offline.plot(
        fig, auto_open=True, filename="saved_graph/category_cooccurence.html"
    )

'saved_graph/category_cooccurence.html'

#### Cooccurrence Graph terms

In [11]:
df_graph_terms = pd.merge(df_indexed, df, left_on = 'docs', right_on = 'description')
df_graph_terms = df_graph_terms[['imdb', 'words']].copy()

fig = network_analysis(
        df_graph_terms,
        variables=["words"],
        key="imdb",
        top_nodes=200,
        global_filter=0,
        n_neighbours=10,
        method="force_directed",
        density=3,
        height_att=1500,
        width_att=1500,
        color="community",
        template="simple_white",
        edge_size=1.5,
        symbol=None,
        save_path=".",
    )

plotly.offline.plot(
        fig, auto_open=True, filename="saved_graph/terms_cooccurence.html"
    )

'saved_graph/terms_cooccurence.html'

#### Create clusters

In [12]:
df_sample = df.sample(9000, random_state = 42)
df_sample = df_sample[df_sample[text_var].notna()].reset_index(drop=True) # cautious not to let any nan

embeddings_reduced, full_embeddings = get_embeddings(
    df_sample,
    field=text_var,
    model="tfidf",
    model_path="distiluse-base-multilingual-cased-v1",#workds if sbert is chosen
)

Embeddings..
Reducing the vectors..
UMAP(angular_rp_forest=True, metric='cosine', n_components=5, n_neighbors=10, verbose=True)
Thu Feb 10 11:01:33 2022 Construct fuzzy simplicial set
Thu Feb 10 11:01:33 2022 Finding Nearest Neighbors
Thu Feb 10 11:01:33 2022 Building RP forest with 10 trees
Thu Feb 10 11:01:36 2022 metric NN descent for 13 iterations
	 1  /  13
	 2  /  13
	 3  /  13
	 4  /  13
	 5  /  13
	 6  /  13
	 7  /  13
	 8  /  13
	 9  /  13
	 10  /  13
	 11  /  13
	Stopping threshold met -- exiting after 11 iterations
Thu Feb 10 11:03:03 2022 Finished Nearest Neighbor Search
Thu Feb 10 11:03:12 2022 Construct embedding


Epochs completed:   0%|            0/500 [00:00]

Thu Feb 10 11:03:41 2022 Finished embedding


#### Clusterize & plot embeddings

In [ ]:
fig = make_bubble(embeddings_reduced, df_sample, text_var= 'description', n_clusters=20)
fig.show()

UMAP Reduction...


#### Create Nested Embeddings & give them names

In [ ]:
# Create Nested clusters.

df_emb = pd.DataFrame(embeddings_reduced)
df_emb['imdb'] = df_sample['imdb']
h_clusters = hierarchical_clusters(df_emb)

In [ ]:
# merge with terms extracted previously
df_enrich = pd.merge(df_sample, df_indexed_full, left_on = text_var, right_on = 'docs')
df_enrich = df_enrich[['imdb', 'main form']].copy()

In [ ]:
# Merge clusters and original dataset
h_clusters_label = pd.merge(h_clusters, df_enrich, on = 'imdb')
h_clusters_label = h_clusters_label.rename(columns = {'main form':'lemma'})
h_clusters_names = hierarchical_clusters_label(h_clusters_label)

#### Make Treemap Diagram

In [ ]:
fig = topics_treemap(nested_topics = h_clusters_names)
plotly.offline.plot(
        fig, auto_open=True, filename="saved_graph/treemap.html"
    )

#### Make Sankey Diagram

In [ ]:
fig = make_sankey(h_clusters_names, field = 'imdb Movies', index_var = 'imdb')
plotly.offline.plot(
        fig, auto_open=True, filename="saved_graph/sankey.html"
    )